# TP5 jointure (SUJET SEUL)
NOM: BOUZOURINE

PRENOM: HICHEM

N Etudiant: 21319982

# Préparation

installer (ou mettre à jour) DuckDB


In [1]:
# duckdb est déjà installé sur les machines de colab
# !pip install duckdb --pre
# !pip install -U duckdb

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.8/17.8 MB 172.0 kB/s eta 0:00:0000:0100:03
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.9.3.dev3968
    Uninstalling duckdb-0.9.3.dev3968:
      Successfully uninstalled duckdb-0.9.3.dev3968

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


Les lib nécessaires pour ce TP

In [2]:
import duckdb

import sqlite3

import time
import pandas as pd

connection au SGBD

In [3]:
db = duckdb.connect('tpch.db')

## Créer les données


On utilise la base de données du benchmark TPC-H.
DuckDB propose la procedure *dbgen* pour générer la base de données.
Les données sont stockées dans le fichier *tpch.db*


Comprendre le [schéma des données TPC-H](https://nuage.lip6.fr/s/RramHt5W3RomySs) (ou cf moodle).

La taille de la base est paramétrée par le scale factor *sf*, vous pouvez faire varier sa valeur de 0.1 à 1 pour créer des bases de plus en plus grandes selon les expériences que vous voulez faire.


In [4]:
# scale factor
sf = 0.2

generate=True

order_table_created = db.execute("select count(*) from duckdb_tables() where table_name = 'orders'").fetchall()[0][0]

if (order_table_created == 1):
  nb_orders = db.execute("select estimated_size from duckdb_tables() where table_name = 'orders'").fetchall()[0][0]
  if nb_orders / 1500000 == sf:
    print("database already generated with sf", sf)
    generate = False

if generate:
  drop_tpch_tables = """
  DROP TABLE IF EXISTS customer;
  DROP TABLE IF EXISTS lineitem;
  DROP TABLE IF EXISTS nation;
  DROP TABLE IF EXISTS orders;
  DROP TABLE IF EXISTS part;
  DROP TABLE IF EXISTS partsupp;
  DROP TABLE IF EXISTS region;
  DROP TABLE IF EXISTS supplier;
  """
  db.execute(drop_tpch_tables)

  print("generate database with scale factor sf = ", sf)
  db.execute(f"CALL dbgen(sf={sf})")

generate database with scale factor sf =  0.2


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# Consulter la BD

## Table names

Le catalogue des tables est duckdb_tables()

Il y a 8 tables pour la base TPCH.

Note: la fonction  `.df()` ajoutée à la fin de l'expression sert à importer le résultat de la requête dans un dataframe pandas afin d'avoir un rendu "ergonomique" dans colab .

In [5]:
db.execute("select table_name, estimated_size, column_count from duckdb_tables() order by estimated_size desc").df()

,table_name,estimated_size,column_count
0,lineitem,1199969,16
1,orders,300000,9
2,partsupp,160000,5
3,part,40000,9
4,customer,30000,8
5,supplier,2000,7
6,nation,25,4
7,region,5,3


 La commande `SHOW TABLES` retourne le nom des tables

In [ ]:
# db.execute("SHOW TABLES").df()

La clause  `LIMIT N` est ajoutée à la fin d'une requête pour calculer seulement les N premiers tuples du résultat.

Exemple pour afficher N tuples de la table Lineitem :

In [6]:
db.execute("SELECT * FROM lineitem LIMIT 5").df()

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,l_shipdate,l_commitdate,l_receiptdate,l_shipinstruct,l_shipmode,l_comment
0,1,31038,1554,1,17.0,16473.51,0.04,0.02,N,O,1996-03-13,1996-02-12,1996-03-22,DELIVER IN PERSON,TRUCK,to beans x-ray carefull
1,1,13462,1463,2,36.0,49516.56,0.09,0.06,N,O,1996-04-12,1996-02-28,1996-04-20,TAKE BACK RETURN,MAIL,according to the final foxes. qui
2,1,12740,741,3,8.0,13221.92,0.10,0.02,N,O,1996-01-29,1996-03-05,1996-01-31,TAKE BACK RETURN,REG AIR,ourts cajole above the furiou
3,1,427,928,4,28.0,37167.76,0.09,0.06,N,O,1996-04-21,1996-03-30,1996-05-16,NONE,AIR,s cajole busily above t
4,1,4806,313,5,24.0,41059.20,0.10,0.04,N,O,1996-03-30,1996-03-14,1996-04-01,NONE,FOB,"the regular, regular pa"


## Describe

La commande `DESCRIBE` retourne une description des attributs d'une table.

### Lineitem

In [7]:
db.execute("DESCRIBE Lineitem").df()

,column_name,column_type,null,key,default,extra
0,l_orderkey,INTEGER,NO,None,None,None
1,l_partkey,INTEGER,NO,None,None,None
2,l_suppkey,INTEGER,NO,None,None,None
3,l_linenumber,INTEGER,NO,None,None,None
4,l_quantity,"DECIMAL(15,2)",NO,None,None,None
5,l_extendedprice,"DECIMAL(15,2)",NO,None,None,None
6,l_discount,"DECIMAL(15,2)",NO,None,None,None
7,l_tax,"DECIMAL(15,2)",NO,None,None,None
8,l_returnflag,VARCHAR,NO,None,None,None
9,l_linestatus,VARCHAR,NO,None,None,None


### Orders

In [13]:
db.execute("DESCRIBE Orders").df()

,column_name,column_type,null,key,default,extra
0,o_orderkey,INTEGER,NO,None,None,None
1,o_custkey,INTEGER,NO,None,None,None
2,o_orderstatus,VARCHAR,NO,None,None,None
3,o_totalprice,"DECIMAL(15,2)",NO,None,None,None
4,o_orderdate,DATE,NO,None,None,None
5,o_orderpriority,VARCHAR,NO,None,None,None
6,o_clerk,VARCHAR,NO,None,None,None
7,o_shippriority,INTEGER,NO,None,None,None
8,o_comment,VARCHAR,NO,None,None,None


### Customer

In [14]:
db.execute("DESCRIBE Customer").df()

,column_name,column_type,null,key,default,extra
0,c_custkey,INTEGER,NO,None,None,None
1,c_name,VARCHAR,NO,None,None,None
2,c_address,VARCHAR,NO,None,None,None
3,c_nationkey,INTEGER,NO,None,None,None
4,c_phone,VARCHAR,NO,None,None,None
5,c_acctbal,"DECIMAL(15,2)",NO,None,None,None
6,c_mktsegment,VARCHAR,NO,None,None,None
7,c_comment,VARCHAR,NO,None,None,None


La commande `SUMMARIZE` donne un aperçu statistique des données d'une table.
Elle retourne pour chaque attribut, les bornes min et max du domaine, le nombre approximatif de valeurs uniques ainsi que la distribution décrite par la moyenne, l'écart type et les quartiles 25, 50 (médiane) et 75.

### Supplier


In [15]:
db.execute("DESCRIBE Supplier").df()

,column_name,column_type,null,key,default,extra
0,s_suppkey,INTEGER,NO,None,None,None
1,s_name,VARCHAR,NO,None,None,None
2,s_address,VARCHAR,NO,None,None,None
3,s_nationkey,INTEGER,NO,None,None,None
4,s_phone,VARCHAR,NO,None,None,None
5,s_acctbal,"DECIMAL(15,2)",NO,None,None,None
6,s_comment,VARCHAR,NO,None,None,None


## Summarize

### Lineitem summary

In [16]:
db.execute("SUMMARIZE lineitem").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,l_orderkey,INTEGER,1,1200000,303168,599740.3752955284,346413.01153687225,299438,597517,901603,1199969,0.0
1,l_partkey,INTEGER,1,40000,39908,20004.208162044186,11548.306002355008,10007,19991,30009,1199969,0.0
2,l_suppkey,INTEGER,1,2000,1991,1001.1636625612829,577.3359391228882,502,1001,1501,1199969,0.0
3,l_linenumber,INTEGER,1,7,7,3.000748352665777,1.7325350842213605,1,3,4,1199969,0.0
4,l_quantity,"DECIMAL(15,2)",1.00,50.00,50,25.52883949502029,14.415643590702027,13,26,38,1199969,0.0
5,l_extendedprice,"DECIMAL(15,2)",901.00,96949.50,219689,36243.27555710189,22145.67713391594,17761,34697,52087,1199969,0.0
6,l_discount,"DECIMAL(15,2)",0.00,0.10,11,0.05006229327590963,0.03161120608482633,0,0,0,1199969,0.0
7,l_tax,"DECIMAL(15,2)",0.00,0.08,9,0.04003356753382796,0.025805399262168755,0,0,0,1199969,0.0
8,l_returnflag,VARCHAR,A,R,3,None,None,None,None,None,1199969,0.0
9,l_linestatus,VARCHAR,F,O,2,None,None,None,None,None,1199969,0.0


### Orders summary

In [8]:
db.execute("SUMMARIZE Orders").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,o_orderkey,INTEGER,1,1200000,303168,599991.5,346410.73887115344,299991,605652,897199,300000,0.0
1,o_custkey,INTEGER,1,29999,20268,14988.668496666667,8657.688778103467,7481,14976,22505,300000,0.0
2,o_orderstatus,VARCHAR,F,P,3,None,None,None,None,None,300000,0.0
3,o_totalprice,"DECIMAL(15,2)",853.00,493724.37,295692,143227.1872945,83943.7695032818,73796,136954,203999,300000,0.0
4,o_orderdate,DATE,1992-01-01,1998-08-02,2404,None,None,None,None,None,300000,0.0
5,o_orderpriority,VARCHAR,1-URGENT,5-LOW,5,None,None,None,None,None,300000,0.0
6,o_clerk,VARCHAR,Clerk#000000001,Clerk#000001000,1012,None,None,None,None,None,300000,0.0
7,o_shippriority,INTEGER,0,0,1,0.0,0.0,0,0,0,300000,0.0
8,o_comment,VARCHAR,Tiresias affix after the silent courts,zzle: slyly even ideas wake furiously across t...,294550,None,None,None,None,None,300000,0.0


### Customer summary

In [17]:
db.execute("SUMMARIZE Customer").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,c_custkey,INTEGER,1,30000,29822,15000.5,8660.398374208891,7500,15000,22500,30000,0.0
1,c_name,VARCHAR,Customer#000000001,Customer#000030000,30933,None,None,None,None,None,30000,0.0
2,c_address,VARCHAR,3SUEwTaEs35ZDr5dx6P2b,zzyQcZpC50YDkK26Pglp,30258,None,None,None,None,None,30000,0.0
3,c_nationkey,INTEGER,0,24,25,12.024033333333334,7.198790395035864,6,12,18,30000,0.0
4,c_phone,VARCHAR,10-101-769-8567,34-999-121-7718,30698,None,None,None,None,None,30000,0.0
5,c_acctbal,"DECIMAL(15,2)",-999.95,9999.72,29815,4458.452009,3174.9204191690947,1715,4415,7224,30000,0.0
6,c_mktsegment,VARCHAR,AUTOMOBILE,MACHINERY,5,None,None,None,None,None,30000,0.0
7,c_comment,VARCHAR,"Tiresias haggle furiously bold, express instr...",zzle. express packages sleep slyly furiously b...,30042,None,None,None,None,None,30000,0.0


### Supplier summary

In [18]:
db.execute("SUMMARIZE Supplier").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,s_suppkey,INTEGER,1,2000,1991,1000.5,577.4945887192364,500,1000,1500,2000,0.0
1,s_name,VARCHAR,Supplier#000000001,Supplier#000002000,2013,None,None,None,None,None,2000,0.0
2,s_address,VARCHAR,"Bg4GNb1 64E,NhHOTBsSLegFZL2l",zw9CeCD4NaTK4vzeCWnQ,2015,None,None,None,None,None,2000,0.0
3,s_nationkey,INTEGER,0,24,25,11.9515,7.166926130851528,6,12,18,2000,0.0
4,s_phone,VARCHAR,10-108-564-6160,34-998-900-4911,2026,None,None,None,None,None,2000,0.0
5,s_acctbal,"DECIMAL(15,2)",-990.13,9993.46,2017,4345.165605,3164.339194624758,1582,4322,7011,2000,0.0
6,s_comment,VARCHAR,above the slyly regular instructions are slyl,ys haggle furiously blithely express asymptote...,1957,None,None,None,None,None,2000,0.0


# Requêtes

## Exécuter une requête

On définit la fonction `run_query` pour exécuter une requête et mesurer sa durée en millisecondes (durée moyenne sur n exécutions)

In [19]:
# par défaut on exécute la requête dans DuckDB (db).
# On execute la requête 3 fois pour avoir une durée moyenne
def run_query(q, connection=db, iteration=3):
  start = time.time()
  for i in range(iteration):
    res = connection.execute(q).fetchall()
    # res = connection.execute(q).arrow()
    # res = connection.execute(q).df()

  end = time.time()
  return(f"{len(res)} tuples(s), {(end - start)*1000/iteration:.0f}ms (avg for {iteration} iter.)")

Exécuter une première requête

In [20]:
query = """
SELECT
    l_linenumber, l_extendedprice, o_orderkey, o_totalprice, c_name
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
AND o_totalprice < 1000
"""

run_query(query)

'17 tuples(s), 19ms (avg for 3 iter.)'

# Plan d'une requête
L'écart de durée entre la requête optimisée et celle non optimisée vient du fait qu'elles ont des plans d'exécution différents

On peut visualiser le **plan** d'exécution d'une requête en préfixant la requête par `EXPLAIN`.

La commande `EXPLAIN` ne fait qu'afficher le plan d'exécution ; elle n'exécute **pas** la requête.

Le plan affiché est appelé *plan physique* car il indique pour chaque opérateur du plan, le nom de l'algorithme utilisé:
*   HASH_GROUP_BY : regroupement par hachage puis agrégation
*   HASH_JOIN : jointure par hachage
*   FILTER : sélection en pipeline
*   SEQ_SCAN : lecture séquentielle d'une table

De plus le plan indique la cardinalité estimée **EC** pour certains opérateurs, en particulier pour les sélections et les jointures.

In [21]:
def explain_query(query):
  print(db.execute("EXPLAIN " + query).fetchall()[0][1])

print(query)
explain_query(query)


SELECT
    l_linenumber, l_extendedprice, o_orderkey, o_totalprice, c_name
FROM
    customer
    JOIN orders ON (c_custkey=o_custkey)
    JOIN lineitem ON (l_orderkey=o_orderkey)
WHERE
    c_mktsegment = 'BUILDING'
AND o_totalprice < 1000

┌───────────────────────────┐                                                          
│         PROJECTION        │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│        l_linenumber       │                                                          
│      l_extendedprice      │                                                          
│         o_orderkey        │                                                          
│        o_totalprice       │                                                          
│           c_name          │                                                          
└─────────────┬─────────────┘                          

Remarque : vous pourriez invoquer `PRAGMA explain_output='all'` pour afficher, en plus du plan physique, le plan logique avant son optimisation et le plan logique optimisé, mais ce n'est pas demandé dans ce TP.

# Rapport d'exécution d'un plan

On peut afficher un rapport **après** l'exécution d'un plan pour connaitre la quantité de données traitées par chaque opération du plan. On appelle cela le *profil* de l'exécution.

La fonction  `run_and_profile_query` active le mode profil afin de sauvegarder les informations de profil dans un fichier.
La requête est ensuite exécutée puis les informations de profil produites pendant l'exécution sont affichées.

On peut lire dans le profil, pour chaque opérateur, la durée écoulée, le nombre de tuples consommés en entrée ainsi que le nombre de tuples produits pour l'opérateur suivant.

Plus spécifiquement, l'opérateur de lecture séquentielle `SEQ_SCAN` (sequential scan) indique le nombre de tuples lus dans une table.
L'opérateur `HASH_GROUP_BY` indique le nombre de groupes créés.

Remarque: pour les opérateurs qui matérialisent leur sortie, le nombre de tuples produits représente la cardinalité du résultat intermédiaire.

Les cardinalités intermédiaires sont importantes car elles peuvent expliquer pourquoi un opérateur a pris beaucoup de temps pour être traité.
Dans de nombreuses situations, il est possible de réduire la cardinalité des résultats intermédiaires et modifiant l'ordre des opérations et la façon dont elles sont traitées.


In [22]:
def run_and_profile_query(query):
  db.execute("PRAGMA enable_profiling")
  db.execute("PRAGMA profiling_output='out.log'")
  # res = db.execute(query).arrow()
  # res = db.execute(query).df()
  res = db.execute(query).fetchall()
  db.execute("PRAGMA disable_profiling")
  print(len(res), "tuple(s)")
  with open('out.log', 'r') as f:
    output = f.read()
  print(output)

run_and_profile_query(query)

17 tuple(s)
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
 SELECT     l_linenumber, l_extendedprice, o_orderkey, o_totalprice, c_name FROM     customer     JOIN orders ON (c_custkey=o_custkey)     JOIN lineitem ON (l_orderkey=o_orderkey) WHERE     c_mktsegment = 'BUILDING' AND o_totalprice < 1000 
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││        Total Time: 0.0215s        ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐                                                          
│      RESULT_COLLECTOR     │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│             0             │                                                      

# Exercice 1 : Jointure par hachage



On considère une requête de jointure entre Lineitem et Orders avec des sélections pour réduire la taille des relations. On a
*   $R = T_1 \bowtie T_2$ avec :
*   $T_1 = \pi_{l\_linenumber} (\sigma_{p_1}(Lineitem))$
*   $T_2 = \pi_{o\_orderkey, o\_custkey} (\sigma_{p_2}(Orders))$

et avec les prédicats de sélection :
*   $p_1: l\_extendedprice \le 902$
*   $p_2: o\_totalprice \ge 1000$




## Question 1: Ecrire R en SQL et mesurer sa durée

In [29]:
query= """
SELECT l_linenumber, o_orderkey, o_custkey
FROM Lineitem
JOIN Orders on (l_orderkey=o_orderkey)
WHERE l_extendedprice <= 902
AND o_totalprice >= 1000

"""

# db.execute("PRAGMA enable_optimizer")
explain_query(query)
print(run_query(query))

# duree .............

┌───────────────────────────┐                             
│         PROJECTION        │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│        l_linenumber       │                             
│         o_orderkey        │                             
│         o_custkey         │                             
└─────────────┬─────────────┘                                                          
┌─────────────┴─────────────┐                             
│         HASH_JOIN         │                             
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                             
│           INNER           │                             
│  l_orderkey = o_orderkey  ├──────────────┐              
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │              │              
│         EC: 49317         │              │              
└─────────────┬─────────────┘              │                                           
┌─────────────┴─────────────┐┌─────────────┴─────────────

## Question 2. Quelles sont les cardinalités estimées et réelles des opérandes  $T_1$ et $T_2$ ?

cardinalité estimée et réelle de T1

In [ ]:
T1 = """
select
from
where
"""

# run_and_profile_query(T1)
# .... tuples , estimation .........

2 tuple(s)
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
 select l_linenumber, l_orderkey from Lineitem where l_extendedprice <= 902 
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││        Total Time: 0.0009s        ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐
│      RESULT_COLLECTOR     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│          lineitem         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│        l_linenumber       │
│         l_orderkey        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ Filters: l_extendedprice< │
│=902.00 AND l_extendedprice│
│  

cardinalité estimée et réelle de T2

In [ ]:
T2 = """
select
from
where
"""
# run_and_profile_query(T2)

# .... tuples , estimation .........

299916 tuple(s)
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
 select o_orderkey, o_custkey from Orders where o_totalprice >= 1000 
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││         Total Time: 0.136s        ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐
│      RESULT_COLLECTOR     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           orders          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         o_orderkey        │
│         o_custkey         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│Filters: o_totalprice>=1000│
│.00 AND o_totalprice IS NOT│
│    

## Question 3 : Jointure en commençant par lire la plus petite relation

On désactive l'optimiseur, afin de modifier l'ordre des relations dans la jointure.



### a) Réécrire R en une requête équivalente pour que :
*    les sélections soient traitées avant les projections et la jointure.
*    La relation la plus petite T1 (car T1 << T2) soit chargée en mémoire avant de lire T2. Donc T2 doit donc être l'opérande de droite de l'opérateur hash join d'après le modèle d'exécution de DuckDB.

(T1 a droite)

In [25]:
query= """
select
from
(  select
   )
JOIN
(  select
   )
on
"""

# db.execute("PRAGMA disable_optimizer")
# explain_query(query)

# print(run_query(query))
# # durée....

b) Est-ce que ce plan est plus rapide que le plan optimisé par DuckDB ?

...........

# Exercice 2 : Jointure par boucles imbriquées avec index

L'objectif est de montrer qu'une jointure par boucles imbriquées avec index peut êter plus rapide pour certaines requêtes. Cet algorithme n'étant pas actuellement implanté dans DuckDB, on propose de l'implanter dans l'application.

### 1) Index sur l'attribut de jointure

a) Dans chaque relation, créer un index sur l'attribut de jointure

In [ ]:
# db.execute("drop index if exists I1;")
# db.execute("create index I1 on ............")

# db.execute("drop index if exists I2;")
# db.execute("create index I2 on ..............")

# # Vérifier la présence des index
# db.execute("select index_name, table_name, is_unique, is_primary, sql from duckdb_indexes()").df()

b) Constater qu'aucun index n'est utilisé dans le plan optimisé. Quelle est la durée d'exécution ?

In [ ]:
query= """
SELECT l_linenumber, o_orderkey, o_custkey
FROM lineitem
JOIN orders on o_orderkey = l_orderkey
WHERE
l_extendedprice <= 902
AND
o_totalprice >= 1000
"""

# db.execute("PRAGMA enable_optimizer")
# explain_query(query)
# print(run_query(query))

c) Quelle est la durée d'exécution de la requête S suivante lorsqu'on utilise l'optimiseur ?

S = $\pi_{o\_custkey}(\sigma_{o\_orderkey=v ~ \texttt{and} ~ o\_totalprice \ge 1000}(Orders))$

In [ ]:
query= """
select
from
where
AND

"""

# db.execute("PRAGMA enable_optimizer")
# explain_query(query)
# print(run_query(query))

┌───────────────────────────┐
│         PROJECTION        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         o_custkey         │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│           FILTER          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ ((o_totalprice >= 1000.00)│
│ AND (o_orderkey = 123456))│
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 60000         │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│        INDEX_SCAN         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           orders          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         o_orderkey        │
│        o_totalprice       │
│         o_custkey         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 300000        │
└───────────────────────────┘                             

(1, '1ms (avg for 10 iter.)')


## 2) Jointure avec index
Ecrire la fonction jointure_avec_index() qui exécute R de la façon suivante:
*   lire T1
*   pour chaque tuple obtenu, exécuter S joindre 2 tuples.

Indication: il est possible de définir une requête "paramétrée" avec un predicat "o_orderkey = ?" et de transmettre la valeur du paramètre au moment de l'exécution. Voir la méthode [execute pour des prepared statements](https://duckdb.org/docs/api/python/dbapi#prepared-statements) .

Mesurer la durée de ce plan. Est-il plus rapide que la requête optimisée ?



In [ ]:
# def jointure_avec_index():
#   res = []
#   query= """
#   select
#   from
#   where
#   """
#   for t in db.execute(query).fetchall():
#     o = db.execute("""
#             select
#             from
#             where  = ?
#             and
#             """, [............]).fetchone()
#     res.append( (*t, *o))
#   return res




# db.execute("PRAGMA enable_optimizer")
# start = time.time()
# res = jointure_avec_index()
# end = time.time()
# print(f"{(end - start)*1000:.0f}ms")
# print(len(res), "tuple(s)")
# print(res)

5ms
2 tuple(s)
[(505280, 4, 27194), (599361, 7, 7657)]


# Exercice 3 : Jointure entre 3 relations

L'objectif est de comprendre la notion d'ordre de jointures.
On sait que pour un ensemble de relations donné, plusieurs ordre de jointure existent.
On étudie différentes requêtes portant sur les mêmes relations. On veut montrer que les plans optimaux (=ceux choisis par l'optimiseur) de ces requêtes n'ont pas tous le même ordre de jointure.

On considère une première requête qui accède aux relations Lineitem, Orders,Customer.

Note: ne pas exécuter cette requête qui est trop longue mais afficher seulement son plan.

In [ ]:
query= """
SELECT l_linenumber, o_orderkey, o_custkey, c_name
FROM Lineitem
JOIN Orders on o_orderkey = l_orderkey
JOIN Customer on o_custkey = c_custkey
"""

# db.execute("PRAGMA enable_optimizer")
# explain_query(query)

┌───────────────────────────┐                                                          
│         PROJECTION        │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│        l_linenumber       │                                                          
│         o_orderkey        │                                                          
│         o_custkey         │                                                          
│           c_name          │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│         HASH_JOIN         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│  


Question: Dans quel ordre les jointures sont-elles traitées ?

Ordre : (lineitem, (orders, customer))

## 1) (Customer, (Orders, Lineitem))

Proposer une requête avec un prédicat de sélection sur Lineitem de telle sorte que le plan de jointure obtenu soit (Customer, (Orders, Lineitem))

In [ ]:
# query= """
# select ...............
# from lineitem
# JOIN orders on o_orderkey = l_orderkey
# JOIN Customer on o_custkey = c_custkey
# where

# """
# explain_query(query)

## 2) (Lineitem, (Customer, Orders))

Proposer une requête avec un prédicat de sélection sur Orders de telle sorte que le plan de jointure obtenu soit (Lineitem, (Customer, Orders))

In [ ]:
# query= """
# select  .............
# from lineitem
# JOIN Orders on o_orderkey = l_orderkey
# JOIN Customer on o_custkey = c_custkey
# where

# """
# explain_query(query)
# print(run_query(query))

## 3)  (Customer, (Lineitem, Orders))

Proposer une requête avec des prédicats de sélection de telle sorte que le plan de jointure obtenu soit (Customer, (Lineitem, Orders))

In [ ]:
# query= """
# select ...........
# from lineitem
# JOIN orders on o_orderkey = l_orderkey
# JOIN Customer on o_custkey = c_custkey
# where

# """
# explain_query(query)
# print(run_query(query))

┌───────────────────────────┐                                                          
│         PROJECTION        │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│        l_linenumber       │                                                          
│         o_orderkey        │                                                          
│         o_custkey         │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│         HASH_JOIN         │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│           INNER           │                                                          
│  

## 4) (LineItem, Orders), Customer)
Est ce qu'une requête sur ces mêmes tables pourrait avoir un plan optimal avec l'ordre (LineItem, Orders), Customer) ?
Justifier.

# Exercice 4 : Implémentation des jointures par hachage

L'objectif de cet exercice est de comprendre le fonctionnement des jointures par hachage : quelles sont les données chargées dans une table de hachage en mémoire, quelles sont les différentes façons de combiner plusieurs jointures par hachage.

Dans toutes les questions où on demande d'implémenter un plan de jointure, les consignes sont:
*    accéder à chaque table avec une requête "mono-table" pouvant contenir seulement des sélections et des projections.
*    charger en mémoire le résultat intermédiaire d'une requête dans un dictionnaire (= une table de hachage).
*    le résultat d'une requête doit être représenté par une liste de tuples.

### 1) Jointures ((Lineitem, Orders), Supplier)
On considère la requête suivante accédant aux relations Lineitem, Orders, Supplier.

Vérifier que l'ordre choisi par l'optimiseur est bien ((Lineitem, Orders), Supplier)

In [ ]:
query= """
SELECT  l_orderkey, l_linenumber, s_name, s_comment, o_totalprice
FROM Lineitem
JOIN Orders on o_orderkey = l_orderkey
JOIN Supplier on l_suppkey = s_suppkey
WHERE o_totalprice < 5000
AND l_extendedprice < 1000
"""
explain_query(query)
print(run_query(query))

┌───────────────────────────┐                                                          
│         PROJECTION        │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│         l_orderkey        │                                                          
│        l_linenumber       │                                                          
│           s_name          │                                                          
│         s_comment         │                                                          
│        o_totalprice       │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│         HASH_JOIN         │                                                          
│  

Définir la fonction query_plan1() qui implémente le plan correspondant

In [ ]:
# def query_plan1():

#   start = time.time()
#   res = []

#   query = "select .......... from ........."
#   dict1 = {....... : ..........  for t in db.execute(query).fetchall()}

#   query = "select ......."
#   dict2 = {....... : ..........  for t in db.execute(query).fetchall()}

#   query = "................."
#   for t in db.execute(query).fetchall():
#     a = dict1.get(..........., None)
#     if(a is not None):
#       ..........
#       ..........
#       ..........
#           res.append(  (........) )

#   end = time.time()
#   return(f"{len(res)} tuples, {(end - start)*1000:.0f}ms")

# print(query_plan1())

83 tuples, 106ms


## 2) Jointure (Orders, (Lineitem, Supplier))

Vérifier que dans la requête suivante, l'ordre des jointures est (Orders, (Lineitem, Supplier))

In [ ]:
query= """
SELECT  l_orderkey, l_linenumber, s_name, s_comment, o_totalprice
FROM Lineitem
JOIN Orders on o_orderkey = l_orderkey
JOIN Supplier on l_suppkey = s_suppkey
WHERE o_totalprice < 5000
AND l_extendedprice < 1000
AND s_nationkey=13
"""
explain_query(query)
print(run_query(query))

┌───────────────────────────┐                                                          
│         PROJECTION        │                                                          
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                          
│         l_orderkey        │                                                          
│        l_linenumber       │                                                          
│           s_name          │                                                          
│         s_comment         │                                                          
│        o_totalprice       │                                                          
└─────────────┬─────────────┘                                                                                       
┌─────────────┴─────────────┐                                                          
│         HASH_JOIN         │                                                          
│  

Définir la fonction query_plan2() qui implémente le plan correspondant

In [ ]:
# def query_plan2():

#   start = time.time()
#   res = []







#   end = time.time()
#   return(f"{len(res)} tuples, {(end - start)*1000:.0f}ms")

# print(query_plan2())

3 tuples, 108ms


# Exercice 5 : Plans non linéaires

Soit la requête de jointure entre les relations Lineitem, Orders, Customer, Supplier

In [ ]:
query= """
select  l_orderkey, l_linenumber
from lineitem
JOIN orders on o_orderkey = l_orderkey
JOIN customer on c_custkey = o_custkey
--JOIN Nation on c_nationkey = n_nationkey
join Supplier on l_suppkey = s_suppkey
where s_acctbal <100

"""
explain_query(query)

┌───────────────────────────┐                                                                                       
│         PROJECTION        │                                                                                       
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                       
│         l_orderkey        │                                                                                       
│        l_linenumber       │                                                                                       
└─────────────┬─────────────┘                                                                                                                    
┌─────────────┴─────────────┐                                                                                       
│         HASH_JOIN         │                                                                                       
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │      

## 1) Quel est l'ordre des jointures ?

L'ordre est ................

## 2) Implémenter le plan

# Divers

In [ ]:
query= """
select  l_orderkey, l_linenumber
from lineitem
JOIN orders on o_orderkey = l_orderkey
JOIN customer on c_custkey = o_custkey
JOIN Nation on c_nationkey = n_nationkey
join Supplier on l_suppkey = s_suppkey
where s_acctbal <100

"""
explain_query(query)

┌───────────────────────────┐                                                                                                                    
│         PROJECTION        │                                                                                                                    
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                                                    
│         l_orderkey        │                                                                                                                    
│        l_linenumber       │                                                                                                                    
└─────────────┬─────────────┘                                                                                                                                                 
┌─────────────┴─────────────┐                                                                  